In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data management (dataframes)
import matplotlib.pyplot as plt  # plotting
import seaborn as sns  # plotting
from scipy import stats # basic statistics
from sklearn.linear_model import LinearRegression
from sklearn.utils import resample  # allows implementing bootstrap
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import BaggingRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor